## Base de datos de artistas de corridos


### Importamos las librerias necesarias

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import pandas as pd

### Hacemos una coneccion con spotify mediante credenciales

In [2]:
load_dotenv()
client_id = os.getenv("CLIENT_ID") #variable de entorno
client_secret = os.getenv("CLIENT_SECRET") #variable de entorno

client_credentials_manager = SpotifyClientCredentials(
    client_id = client_id,
    client_secret = client_secret
)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Creamos una lista con el id de cada artista

In [3]:
Natanael_Cano_uri = 'spotify:artist:0elWFr7TW8piilVRYJUe4P'
Peso_Pluma_uri = 'spotify:artist:12GqGscKJx3aE4t07u7eVZ'
Junior_H_uri = 'spotify:artist:7Gi6gjaWy3DxyilpF1a8Is'
Oscar_Maydon_uri = 'spotify:artist:3l9G1G9MxH6DaRhwLklaf5'
Fuerza_Regida_uri = 'spotify:artist:0ys2OFYzWYB5hRDLCsBqxt'
Tito_Double_P_uri = 'spotify:artist:5eumcnUkdmGvkvcsx1WFNG'
Gabito_Ballesteros_uri = 'spotify:artist:6Sbl0NT50roqWvy746MfVf'
Chino_Pacas_uri = 'spotify:artist:2rmkQLzj0k4nZdQehOUByO'

Artists = [Natanael_Cano_uri, Peso_Pluma_uri, Junior_H_uri, Oscar_Maydon_uri, Fuerza_Regida_uri, 
           Tito_Double_P_uri, Gabito_Ballesteros_uri, Chino_Pacas_uri]

### Obtencion de los datos

Iteramos sobre cada artista, para luego volver a iterar sobre cada album y sobre cada cancion del album. Guardamos los items de cada categoria en una lista.

In [4]:
artist_list = []
albums_list = [] 
songs_list = []
for artist in Artists:   
    artist_results = sp.artist(artist)
    artist_info ={
        'artist_name': artist_results['name'],
        'artist_id': artist_results['id'],
        'image_artist': artist_results['images'][0]['url'] if artist_results['images'] else None}  
    artist_list.append(artist_info)  

    albums_results = sp.artist_albums(artist, album_type='album')
    albums = albums_results['items']
    while albums_results['next']:
        albums_results = sp.next(albums_results)
        albums.extend(albums_results['items'])

    #Crear lista de diccionarios para el DataFrame 
    for album in albums: 
        album_info = {
            'album_id': album['id'], 
            'album_name': album['name'], 
            'artist_id': artist_results['id'],
            'total_tracks': album['total_tracks'], 
            'image_album': album['images'][0]['url'] if album['images'] else None, 
            'release_date_album': album['release_date']} 
        albums_list.append(album_info) 

        songs_results = sp.album_tracks(album['id'], limit =50)
        for i in range(0,album['total_tracks']):
            songs_info = {
                'song_id': songs_results['items'][i]['id'], 
                'name_song': songs_results['items'][i]['name'], 
                'track_number': songs_results['items'][i]['track_number'],
                'album_id': album['id'],
                'album_name': album['name']} 
            songs_list.append(songs_info)

### Convertimos cada lista a un dataframe y luego guardamos como archivo csv

In [5]:
df_artist = pd.DataFrame(artist_list)
df_albums = pd.DataFrame(albums_list) 
df_songs = pd.DataFrame(songs_list) 

df_artist.to_csv('artists.csv', index = False)
df_albums.to_csv('albums.csv', index=False) 
df_songs.to_csv('songs.csv', index = False)

## Resultados

### Dataframe de artistas

In [6]:
df_artist.head()

,artist_name,artist_id,image_artist
0,Natanael Cano,0elWFr7TW8piilVRYJUe4P,https://i.scdn.co/image/ab6761610000e5eb7e2078...
1,Peso Pluma,12GqGscKJx3aE4t07u7eVZ,https://i.scdn.co/image/ab6761610000e5ebe5283f...
2,Junior H,7Gi6gjaWy3DxyilpF1a8Is,https://i.scdn.co/image/ab6761610000e5eb50998d...
3,Oscar Maydon,3l9G1G9MxH6DaRhwLklaf5,https://i.scdn.co/image/ab6761610000e5eb505626...
4,Fuerza Regida,0ys2OFYzWYB5hRDLCsBqxt,https://i.scdn.co/image/ab6761610000e5eb7e2545...


### Dataframe de albums

In [7]:
df_albums.head()

,album_id,album_name,artist_id,total_tracks,image_album,release_date_album
0,507VZINfvVEAyrcPbJuOVU,Nata Montana,0elWFr7TW8piilVRYJUe4P,15,https://i.scdn.co/image/ab67616d0000b273e2e093...,2023-06-30
1,2irmTvrbuG1ox2Qa6R8J1L,NataKong,0elWFr7TW8piilVRYJUe4P,18,https://i.scdn.co/image/ab67616d0000b273e86670...,2022-04-08
2,64VKaBv6HCauMYY0MOOZof,Soy El Diablo,0elWFr7TW8piilVRYJUe4P,12,https://i.scdn.co/image/ab67616d0000b273c30dfe...,2022-01-20
3,4TATXEH2tnZVTqlyURoWUR,A Mis 20,0elWFr7TW8piilVRYJUe4P,11,https://i.scdn.co/image/ab67616d0000b2738ba654...,2021-05-28
4,5Z94qF1B7RtrwSPUG9dRLd,Nata,0elWFr7TW8piilVRYJUe4P,8,https://i.scdn.co/image/ab67616d0000b2732e0f84...,2021-04-16


### Dataframe de canciones

In [8]:
df_songs

,song_id,name_song,track_number,album_id,album_name
0,57pe0wF0RgVPtcR4f1eXH2,Diran De Mi,1,507VZINfvVEAyrcPbJuOVU,Nata Montana
1,4Zx0yrZwW4uNpd1FhIrOV5,Mas Altas Que Bajadas,2,507VZINfvVEAyrcPbJuOVU,Nata Montana
2,3A2UK3SykAxeo3jeQv9ekR,La Lokerona,3,507VZINfvVEAyrcPbJuOVU,Nata Montana
3,0y6i1ZmaMj3jD6UxN0Aluu,Un Convoy,4,507VZINfvVEAyrcPbJuOVU,Nata Montana
4,7ctRhcAiTBQLsH64s93sN4,Pancake,5,507VZINfvVEAyrcPbJuOVU,Nata Montana
...,...,...,...,...,...
670,6SvAvDIzS5ZWBagkc7XOgp,El Gordo Trae El Mando,11,0IeCqguWKSslODbwnrelhq,Que Sigan Llegando Las Pacas (Extended)
671,1QNgzbXdq1X20vZQsUVSuN,Otra Vez Pegue Un Vergazo,12,0IeCqguWKSslODbwnrelhq,Que Sigan Llegando Las Pacas (Extended)
672,0x1ZJ14BY5inHN9RsSllDO,La Trokita (with Los Nuevos Rebeldes),13,0IeCqguWKSslODbwnrelhq,Que Sigan Llegando Las Pacas (Extended)
673,1de69KbI8CMGvANmoH3970,Que Tranza,14,0IeCqguWKSslODbwnrelhq,Que Sigan Llegando Las Pacas (Extended)


### Filtramos por album

In [9]:
df_songs.loc[df_songs.loc[:, 'album_name'] == 'Nata Montana']

,song_id,name_song,track_number,album_id,album_name
0,57pe0wF0RgVPtcR4f1eXH2,Diran De Mi,1,507VZINfvVEAyrcPbJuOVU,Nata Montana
1,4Zx0yrZwW4uNpd1FhIrOV5,Mas Altas Que Bajadas,2,507VZINfvVEAyrcPbJuOVU,Nata Montana
2,3A2UK3SykAxeo3jeQv9ekR,La Lokerona,3,507VZINfvVEAyrcPbJuOVU,Nata Montana
3,0y6i1ZmaMj3jD6UxN0Aluu,Un Convoy,4,507VZINfvVEAyrcPbJuOVU,Nata Montana
4,7ctRhcAiTBQLsH64s93sN4,Pancake,5,507VZINfvVEAyrcPbJuOVU,Nata Montana
5,1UwCSHDl10Hga2jswZ6plN,Viejo Lobo,6,507VZINfvVEAyrcPbJuOVU,Nata Montana
6,7Fxb0ErkBj37nNQwmIhbRq,AFS,7,507VZINfvVEAyrcPbJuOVU,Nata Montana
7,3FUVgc4ULS0t7X7sUBZ1UO,Eres,8,507VZINfvVEAyrcPbJuOVU,Nata Montana
8,4t46soaqA758rbukTO5pp1,Mi Bello Angel,9,507VZINfvVEAyrcPbJuOVU,Nata Montana
9,6PlLiHoRcjCgK5som2T3ph,En Corto,10,507VZINfvVEAyrcPbJuOVU,Nata Montana
